In [1]:
import requests
import pandas as pd
import ast
import json
import urllib
import tqdm
import os

pd.set_option('display.max_columns', None)

def get_reviews(appid, params):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url + appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        if response.status_code != 200 or response.json()["success"] not in [1, "1"]:
            print("LOG: ERROR", response.status_code)
            raise ValueError("Status code != 200 or query unsuccessful.")
        return response.json()
    
def get_n_reviews(appid, n):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'updated',
            'language' : 'all',
            'day_range' : 365,
            'review_type' : 'all',
            'purchase_type' : 'all',
            }
    total = False
#     with tqdm.tqdm() as pbar:
    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']
#         pbar.update(len(response['reviews']))

        if not total:
                print("total", response["query_summary"]["total_reviews"])
                total = True

        if response["query_summary"]["num_reviews"] == 0:
            break

    return reviews

In [2]:
# download https://www.kaggle.com/datasets/fronkongames/steam-games-dataset
# unpack in the same dir as this notebook (should be a 'games.csv' file inside)

In [3]:
games_df = pd.read_csv("games.csv").sample(frac=1, random_state=42 # DO NOT CHANGE!
                                          ).reset_index(drop=True)
print(games_df.shape)
games_df.head(10)

(76987, 39)


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,1936000,The Infernal Return,"Sep 10, 2022",0 - 0,0,0,6.59,0,Overview The design of the environment is base...,"['English', 'Simplified Chinese', 'Traditional...","['English', 'Traditional Chinese']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://knifish.wixsite.com/games,https://knifish.wixsite.com/games,knifish@mail.com,True,False,False,0,NaN,0,0,0,NaN,0,0,You will rarely but not never see some male sk...,0,0,0,0,Knifish Games,"Knifish Studio,Angel Star Studios","Single-player,Full controller support","Action,Adventure,RPG,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,425210,Shadwen,"May 17, 2016",100000 - 200000,4,0,16.99,0,Shadwen is a 3rd person stealth-action game wh...,"['English', 'French', 'Italian', 'German', 'Sp...","['English', 'Spanish - Spain']",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.shadwen.com,http://www.frozenbyte.com/contact/#Support,support@frozenbyte.com,True,True,True,53,https://www.metacritic.com/game/pc/shadwen?fta...,0,888,280,NaN,48,893,NaN,411,0,613,0,Frozenbyte,Frozenbyte,"Single-player,Steam Achievements,Full controll...","Action,Adventure","Action,Stealth,Adventure,Female Protagonist,As...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,2151890,Wild Isles Playtest,"Sep 14, 2022",0 - 0,0,0,0.00,0,NaN,[],[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,False,False,0,NaN,0,0,0,NaN,0,0,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,719130,OVO Smash!,"Oct 10, 2017",0 - 20000,0,0,5.99,0,What happens when Memory Match meets Deathmatc...,"['English', 'French', 'Italian', 'German', 'Sp...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.inprincipleinc.com/games.html,NaN,support@inprincipleinc.com,True,False,False,0,NaN,0,8,1,NaN,50,0,NaN,0,0,0,0,in Principle Games,in Principle Inc,"Single-player,Multi-player,PvP,Shared/Split Sc...","Casual,Indie","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,891520,Tranquil Garden,"Sep 18, 2020",0 - 20000,0,0,0.00,0,Can the World save itself? In a serene setting...,['English'],[],“Tranquil Garden could set a new standard and ...,https://cdn.akamai.steamstatic.com/steam/apps/...,https://glassrobotgames.com/tranquil-garden,glassrobotgames.com,GlassRobotGames@Gmail.com,True,False,False,0,NaN,0,5,1,NaN,0,0,NaN,0,0,0,0,Glass Robot,Glass Robot,"Single-player,Partial Controller Support","Adventure,Casual,Indie,RPG,Strategy","RPG,Indie,Casual,Adventure,Strategy",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
5,1448650,Close Contact,"Nov 2, 2020",20000 - 50000,0,0,4.99,0,"Close Contact is a short (approx 1-1.5 hours),...",['English'],['English'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,https://twitter.com/ryanjavanshir,ryanjavanshir@hotmail.co.uk,True,False,False,0,NaN,0,13,3,NaN,0,0,NaN,0,0,0,0,Ryan Javanshir,Ryan Javanshir,Single-player,"Action,Adventure,Indie","Action,Adventure,Horror,Exploration,Survival,F...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
6,786570,The Alpha Device,"Feb 13, 2018",20000 - 50000,0,0,0.00,0,"Beyond space, beyond reason... beyond humanity...",['English'],['English'],“When you find a monolith you’ll discover part...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://xiotex-studio

In [4]:
DIR = "steam_reviews_v2"
if not os.path.exists(DIR):
    os.mkdir(DIR)

In [ ]:
MAX_REVIEWS_PER_GAME = 1e9 # DO NOT CHANGE!
IDXS = {
    "artemiy_1": (0, 7_000),
    "artemiy_2": (7_000, 14_000),
    "artemiy_3": (14_000, 20_000),
    "artem_1": (20_000, 27_000),
    "artem_2": (27_000, 34_000),
    "artem_3": (34_000, 40_000),
    "arina_1": (40_000, 47_000),
    "arina_2": (47_000, 54_000),
    "arina_3": (54_000, 60_000),
    "assol_1": (60_000, 67_000),
    "assol_2": (67_000, 74_000),
    "assol_3": (74_000, games_df.shape[0])}
START, END = IDXS["artemiy_1"] # your name here
for i, row in games_df.loc[START: END].iterrows():
    break
    try:
        reviews = get_n_reviews(str(row.AppID), n=MAX_REVIEWS_PER_GAME)
        print(len(reviews), row.Name)

        reviews_df = pd.json_normalize(reviews)
        reviews_df.columns = [column.replace(".", "_") for column in reviews_df.columns]
        reviews_df.to_csv(os.path.join(DIR, f"{row.AppID}.csv"), index=False)
    except KeyboardInterrupt:
        print("Stopping.")
        break
    except:
        print("Encountered exception at", row.AppID)